In [1]:
import re
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
node = []
allDatas = []
siganlList = []
SignalsName = []
messageName = []

In [2]:
def readFile(file_name):
    global node,allDatas,siganlList,SignalsName,messageName
    ''' 得到dbc文件的绝对路径'''
    filePath = file_name
    if filePath:
        print(filePath)
        f = open(filePath, "r")  # 设置文件对象
    else:
        print("读取文件失败！")
        return 0
    """
    NodesPattern:节点
    MessagePattern：消息
    SignalPattern：信号
    """
    NodesPattern = re.compile(r"BU_: (.*)", re.S)
    MessagePattern = re.compile(r"BO_ (.*?) (.*?): (.*?) (.*)", re.S)
#     SignalPattern = re.compile('''SG_ (.*?) : (.*?)\|(.*?)@.*? \((.*?),(.*?)\) \[(.*?)\|(.*?)\] "(.*?)" (.*)''', re.S)
    SignalPattern = re.compile('''SG_ (.*?) : (.*?)\|(.*?)@([0-9])([+|-]) \((.*?),(.*?)\) \[(.*?)\|(.*?)\] "(.*?)" (.*)''', re.S)
    DefaultValue = '''BA_ "GenSig(.*?)" SG_ (\d+) signalname (\d+);'''

    line = f.readline()
    allDatas=[]
    while line:
        """ 匹配出节点 """
        NodesSearched = re.search(NodesPattern, line.strip())
        if NodesSearched:
            node = NodesSearched.group(1).split(" ")
            #print(node)
        """ 匹配出消息 """
        MessageSearched = re.search(MessagePattern, line.strip())
        if MessageSearched:
            siganlList.clear()
            """如果匹配到了message，则获取到message的相关参数 
             比如匹配到了NM_Message_ESC_409，则会解析出改message的一些参数构成list对象['1033', 'NM_Message_ESC_409', '8', 'ESC']
             这四个参数分别是 messgage ID ;message name ; messgae dataLenth ,message sender
             而且把这个list对象 加在了 siganlList 索引0的位置
            """
            Message = list(MessageSearched.groups())
            siganlList.append(Message)
            """ 只 要 message的名字 messageName 列表中"""
            messageName.append(Message[1])
            """读取下一行"""
            line = f.readline()
            """因为有些message并没有定义signal，所以 下一行还是message"""
            MessageSearched = re.search(MessagePattern, line.strip())
            SignalSearched = re.search(SignalPattern, line.strip())
            """下一行如果不是message的内容 就一定是signal的内容了"""
            if not MessageSearched:
                while SignalSearched:
                    """获取信号的参数追加到siganlList"""
                    signal = list(SignalSearched.groups())
                    siganlList.append(signal)
                    """只获取 signal name"""
                    SignalsName.append(signal[0])

                    # 再次解析信号，直到这个message下的信号全部解析完毕
                    line = f.readline()
                    SignalSearched = re.search(SignalPattern, line.strip())
           # print(siganlList)
            c = copy.deepcopy(siganlList)
            allDatas.append(c)
        else:
            line = f.readline()
            MessageSearched = re.search(MessagePattern, line.strip())
    f.close()  # 将文件关闭
    return 1

In [3]:
import os
file_names = []
# path = os.path.abspath('../My_DBC')
path = os.path.abspath('../opendbc-master/opendbc-master-delFalseData')
# path = os.path.abspath('./opendbc-master/opendbc-master')
dirs = os.listdir(path)                    # 获取指定路径下的文件
for i in dirs:
    if os.path.splitext(i)[1] == ".dbc":
        file_names.append(os.path.join(path,i))
# file_names

In [4]:
def get_id_labels(index):
    file_name = file_names[index]
    readFile(file_name)

    # 将DBC文件中的ID(10进制数)改为ID(16进制数)
    data_gts = {}
    for alldata in allDatas:
        data_gts[str(hex(int(alldata[0][0])))[2:]] = alldata

    # 得到DBC文件中各ID Message对应信号位置
    all_id_lsbs = {}
    for k in data_gts.keys():
        message = data_gts[k]
        lsb_list = []
        message_id = message[0][0]+"_"+message[0][1]
        del message[0]
        for signal in message:
            start = int(signal[1])
            length = int(signal[2])
            endianness = int(signal[3])
            start_row = int(start / 8)
            start_col = 8 - (start - start_row * 8) - 1
            new_start = start_row * 8 + start_col
            if endianness == 0:
                new_end = new_start + length
                cur_lsb = new_end - 1
            if endianness == 1:
                cur_lsb = new_start
            lsb_list.append(cur_lsb)
        all_id_lsbs[k] = lsb_list
    return all_id_lsbs

In [5]:
def get_signal_positions(index):
    file_name = file_names[index]
    readFile(file_name)
    
    data_gts = {}
    for alldata in allDatas:
        data_gts[str(hex(int(alldata[0][0])))[2:]] = alldata
        
    signal_positions = {}
    flag = False
    for k in data_gts.keys():
        message = data_gts[k]
        matrix = [[0] * 8 for _ in range(8)]
        message_id = message[0][0]+"_"+message[0][1]
        del message[0]
        cur_signal_positions = []
        for signal in message:
            start = int(signal[1])
            length = int(signal[2])
            endianness = int(signal[3])
            start_row = int(start / 8)
            start_col = 8 - (start - start_row * 8) - 1
            new_start = start_row * 8 + start_col
            if endianness == 0:
                new_end = new_start + length
                cur_msb = new_start
                cur_lsb = new_end - 1
            if endianness == 1:
                cur_lsb = new_start
                count = 0
                x = start_row
                y = start_col
                while count < (length-1):
                    if y == 0:
                        y = 7
                        x = x + 1
                    else:
                        y = y - 1
                    count = count + 1
                cur_msb = x * 8 + y
            cur_signal_positions.append((cur_msb, cur_lsb))
        signal_positions[k] = cur_signal_positions
    return signal_positions

In [6]:
def get_label01(all_id_lsbs):
    # 将各ID Message的信号位置改为 0/1 label
    all_id_labels = {}
    for k in all_id_lsbs:
        cur_lsb_list = all_id_lsbs[k]
        cur_label = np.zeros(64)
        for cur_lsb in cur_lsb_list:
            cur_label[cur_lsb] = 1
        all_id_labels[k] = cur_label
    return all_id_labels

In [61]:
def get_labels(all_id_lsbs):
    # 将各ID Message的信号位置改为 0/1 label
    all_id_labels = {}
    for k in all_id_lsbs:
        cur_lsb_list = all_id_lsbs[k]
        cur_label = [2] * 64
        for cur_lsb in cur_lsb_list:
            cur_label[cur_lsb] = 1
        all_id_labels[k] = cur_label
    return all_id_labels

In [8]:
def get_trace(data):
    group = data.groupby(by="ID")
    id_tracedict = {}
    for trace in list(group):
        id_tracedict[trace[0]] = trace[1]["bin"].apply(lambda x: list(x)).apply(lambda x: list(map(int, x))).tolist()
    return id_tracedict

In [9]:
def get_ids(id_tracedict_int, all_id_labels):
    ids = []
    for id in id_tracedict_int.keys():
        if(id_tracedict_int.__contains__(id) and all_id_labels.__contains__(id)):
            ids.append(id)
    return ids

In [10]:
import random
# 设置全局的随机种子
random.seed(100)
# 以id为单位shuffle
def get_train_test_ids(id_tracedict, all_id_labels):
    ids = get_ids(id_tracedict, all_id_labels)
    random.shuffle(ids)
    train_len = int(len(ids)*0.9)
    train_ids = ids[:train_len]
    test_ids = ids[train_len:]
    return train_ids, test_ids

In [11]:
def get_trace_data(id_tracedict_int, all_id_labels): 
    cur_trace_data = []
    cur_trace_label = []
    for id in id_tracedict_int.keys():
        cur_trace = id_tracedict_int[id]
        cur_label = all_id_labels[id]
        length = len(cur_trace)
        start = 4000
        end = start + 1000
        n = int((length-4000)/1000)
        for i in range(n):
            cur_trace_data.append(cur_trace[start: end])
            cur_trace_label.append(cur_label)
            start = end
            end = start + 1000
    return cur_trace_data, cur_trace_label

In [12]:
def get_data_label(id_tracedict_int, all_id_labels):
    cur_trace_data, cur_trace_label = get_trace_data(id_tracedict_int, all_id_labels)
    trace_indexs = pd.DataFrame(cur_trace_label).index.tolist()
    random.shuffle(trace_indexs)
    trace_data, trace_labels = np.array(cur_trace_data)[trace_indexs], np.array(cur_trace_label)[trace_indexs]
    return trace_data, trace_labels

In [13]:
def get_trace_data_dict(cur_ids, id_tracedict_int, all_id_labels, trace_data_dict, trace_label_dict): 
    for id in cur_ids:
        trace_data_dict[id] = id_tracedict_int[id]
        trace_label_dict[id] = all_id_labels[id]

In [14]:
honda_dbcs = []
cadillac_dbcs = []
tesla_dbcs = []
gm_dbcs = []
for i in range(len(file_names)):
    cur_name = file_names[i].split("\\")[-1].split("_")[0]
    if cur_name == "honda":
        honda_dbcs.append(i)
    if cur_name == "cadillac":
        cadillac_dbcs.append(i)
    if cur_name == "tesla":
        tesla_dbcs.append(i)
    if cur_name == "gm":
        gm_dbcs.append(i)

In [15]:
honda_id_labels = {}
for cur_dbc_index in honda_dbcs:
    honda_id_labels.update(get_label01(get_id_labels(cur_dbc_index))) 
# print(len(honda_id_labels)) 

cadillac_id_labels = {}
for cur_dbc_index in cadillac_dbcs:
    cadillac_id_labels.update(get_label01(get_id_labels(cur_dbc_index)))  
for cur_dbc_index in gm_dbcs:
    cadillac_id_labels.update(get_label01(get_id_labels(cur_dbc_index))) 
# print(len(cadillac_id_labels)) 

# chevy_id_labels = {}
# for cur_dbc_index in gm_dbcs:
#     chevy_id_labels.update(get_label01(get_id_labels(cur_dbc_index))) 
# print(len(chevy_id_labels)) 

D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\opendbc-master\opendbc-master-delFalseData\honda_accord_2018_can_generated.dbc
D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\opendbc-master\opendbc-master-delFalseData\honda_civic_hatchback_ex_2017_can_generated.dbc
D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\opendbc-master\opendbc-master-delFalseData\honda_civic_sedan_16_diesel_2019_can_generated.dbc
D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\opendbc-master\opendbc-master-delFalseData\honda_civic_touring_2016_can_generated.dbc
D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\opendbc-master\opendbc-master-delFalseData\honda_clarity_hybrid_2018_can_generated.dbc
D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\opendbc-master\opendbc-master-delFalseData\honda_crv_executive_2016_can_generated.dbc
D:\--storage--\program\Code\jupyter-notebook\CarNet

In [16]:
# 得到Honda车的子数据集数据
data_hd = pd.read_csv('../Data/Honda/honda001.csv')
data_hd["bin"] = data_hd["Data"].apply(int, base=16).apply(bin).str[2:].apply(lambda x: x.zfill(64))

# 得到Honda车子的数据集的大Trace dict
id_tracedict_hd = get_trace(data_hd)

# 得到用于训练和测试的id
train_ids_hd, test_ids_hd = get_train_test_ids(id_tracedict_hd, honda_id_labels)

In [17]:
# 得到Cadillac车的子数据集数据
data_cd = pd.read_csv('../Data/Cadillac/CadillacCSV/001.txt.csv')
data_cd["bin"] = data_cd["Data"].apply(int, base=16).apply(bin).str[2:].apply(lambda x: x.zfill(64))

# 得到Honda车子的数据集的大Trace dict
id_tracedict_cd = get_trace(data_cd)

# 得到用于训练和测试的id
train_ids_cd, test_ids_cd = get_train_test_ids(id_tracedict_cd, cadillac_id_labels)

In [18]:
# 得到honda和caddilac车的trace dict及对应的label dict
train_data_dict = {}
train_label_dict = {}
get_trace_data_dict(train_ids_hd, id_tracedict_hd, honda_id_labels, train_data_dict, train_label_dict)
get_trace_data_dict(train_ids_cd, id_tracedict_cd, cadillac_id_labels, train_data_dict, train_label_dict)
# get_trace_data_dict(train_ids_ch, id_tracedict_ch, chevy_id_labels, train_data_dict, train_label_dict)

test_data_dict = {}
test_label_dict = {}
get_trace_data_dict(test_ids_hd, id_tracedict_hd, honda_id_labels, test_data_dict, test_label_dict)
get_trace_data_dict(test_ids_cd, id_tracedict_cd, cadillac_id_labels, test_data_dict, test_label_dict)
# get_trace_data_dict(test_ids_ch, id_tracedict_cd, cadillac_id_labels, test_data_dict, test_label_dict)

In [19]:
# 使用READ方法计算bit-flip
def get_bit_Flips(cur_id_traces):
    bit_Flips = []
    for i in range(len(cur_id_traces)):
        messagelist = cur_id_traces[i]
        trace_len = len(messagelist)
        bit_flip = np.zeros(64)
        previous = messagelist[0]
        for item in messagelist:
            for ix in range(64):
                if item[ix] != previous[ix]:
                    bit_flip[ix] = bit_flip[ix] + 1
            previous = item
        for ix in range(64):
            bit_flip[ix] = bit_flip[ix] / trace_len
        bit_flip = np.append(bit_flip, 0)
        bit_flip = np.append(bit_flip, 0)
        bit_Flips.append(bit_flip)
    return bit_Flips

In [20]:
# 先删除constant bit，然后，将trace序列拼成64。问题：会破环bit之间的相关性
def del_constant_bit_concate(cur_trace_data, cur_trace_label, bit_Flips):
    all_trace_data = []
    all_trace_label = []
    for i in range(len(bit_Flips)):
        cur_bit_flip = bit_Flips[i]
        cur_constant_bits = []
        for j in range(64):
            if cur_bit_flip[j] == 0:
                cur_constant_bits.append(j)
        cur_trace = []
        for temp in cur_trace_data[i]:
            cur_trace.append([temp[j] for j in range(len(temp)) if j not in cur_constant_bits])
        cur_label = []
        for j in range(len(cur_trace_label[i])):
            next_j = j+1
            if j not in cur_constant_bits:
                if next_j in cur_constant_bits:
                    cur_label.append(1)
                else:
                    cur_label.append(cur_trace_label[i][j])   
        if i == 0:
            all_trace_data = np.array(cur_trace)
            all_trace_label = np.array(cur_label)
        else:
            all_trace_data = np.hstack((all_trace_data, np.array(cur_trace)))
            all_trace_label = np.hstack((all_trace_label, np.array(cur_label)))
    
    
    num = int(all_trace_data.shape[1]/64)
    cur_trace_data = np.array(np.array_split(all_trace_data[:,0:num*64], num, axis=1))
    cur_trace_label = np.array(np.array_split(all_trace_label[0:num*64], num, axis=0))
    return cur_trace_data, cur_trace_label

In [21]:
def fill_list(cur_list, number, length):
    for i in range(len(cur_list),length):
        cur_list.append(number)
    return cur_list

In [22]:
# 只删除constant bit，形成不定长的trace 序列
def del_constant_bit(cur_trace_data, cur_trace_label, bit_Flips):
    all_trace_data = []
    all_trace_label = []
    for i in range(len(bit_Flips)):
        cur_bit_flip = bit_Flips[i]
        cur_constant_bits = []
        for j in range(64):
            if cur_bit_flip[j] == 0:
                cur_constant_bits.append(j)
        cur_trace = []
        for temp in cur_trace_data[i]:
            res_trace = [temp[j] for j in range(len(temp)) if j not in cur_constant_bits]
            cur_trace.append(fill_list(res_trace, -1, 64))
        cur_label = []
        for j in range(len(cur_trace_label[i])):
            next_j = j+1
            if j not in cur_constant_bits:
                if next_j in cur_constant_bits:
                    cur_label.append(1)
                else:
                    cur_label.append(cur_trace_label[i][j])  
        cur_label = fill_list(cur_label, -1, 64)
        all_trace_data.append(cur_trace)
        all_trace_label.append(cur_label)
    return np.array(all_trace_data), np.array(all_trace_label)

In [23]:
# 只删除constant bit，形成不定长的trace 序列
def del_constant_bit_new(cur_trace_data, cur_trace_label, bit_Flips):
    all_trace_data = []
    all_trace_label = []
    for i in range(len(bit_Flips)):
        cur_bit_flip = bit_Flips[i]
        cur_constant_bits = []
        for j in range(64):
            if cur_bit_flip[j] == 0:
                if j == 0 or cur_bit_flip[j-1] == 0:
                    cur_constant_bits.append(j)
        cur_trace = []
        for temp in cur_trace_data[i]:
            res_trace = [temp[j] for j in range(len(temp)) if j not in cur_constant_bits]
            cur_trace.append(fill_list(res_trace, -1, 64))
        cur_label = []
        for j in range(len(cur_trace_label[i])):
            next_j = j+1
            if j not in cur_constant_bits:
                if cur_bit_flip[j] != 0 and cur_bit_flip[next_j] == 0:
                    cur_label.append(1)
                else:
                    cur_label.append(cur_trace_label[i][j])  
        cur_label = fill_list(cur_label, -1, 64)
        all_trace_data.append(cur_trace)
        all_trace_label.append(cur_label)
    return np.array(all_trace_data), np.array(all_trace_label)

In [62]:
def change_labels(cur_trace_data, cur_trace_label, bit_Flips):
    for i in range(len(bit_Flips)):
        cur_bit_flip = bit_Flips[i]
        for j in range(64):
            if cur_bit_flip[j] == 0:
                cur_trace_label[i][j] = 0
    return cur_trace_data, cur_trace_label

In [27]:
# shuffle后得到训练集和测试集，其中训练集的id占70%，测试集的id占30%
train_data, train_labels = get_data_label(train_data_dict, train_label_dict)
bit_Flips_train = get_bit_Flips(train_data)

In [28]:
# train_data, train_labels = change_labels(train_data, train_labels, bit_Flips_train)
train_data_del, train_labels_del = del_constant_bit(train_data, train_labels, bit_Flips_train)
train_data_del_new, train_labels_del_new = del_constant_bit_new(train_data, train_labels, bit_Flips_train)

In [29]:
test_data, test_labels = get_data_label(test_data_dict, test_label_dict)
bit_Flips_test = get_bit_Flips(test_data)

In [30]:
# test_data, test_labels = change_labels(test_data, test_labels, bit_Flips_test)
test_data_del, test_labels_del = del_constant_bit(test_data, test_labels, bit_Flips_test)
test_data_del_new, test_labels_del_new = del_constant_bit_new(test_data, test_labels, bit_Flips_test)

In [45]:
train_data_del_lstm = train_data_del.transpose(0,2,1)
test_data_del_lstm = test_data_del.transpose(0,2,1)

In [46]:
train_data_lstm = train_data.transpose(0,2,1)
test_data_lstm = test_data.transpose(0,2,1)

In [34]:
import keras
from keras.models import Model, Sequential
from keras.layers import Dense, Activation, Dropout, LSTM, Input, Permute, TimeDistributed, Bidirectional, BatchNormalization, Flatten, MaxPooling1D, Conv1D, concatenate, Masking, Lambda
from keras.losses import mean_absolute_percentage_error, mean_absolute_error
from sklearn.metrics import mean_squared_error
import keras.backend as K
from sklearn import metrics
from keras_contrib.layers import CRF
from keras import losses

In [98]:
def create_model():
    model = Sequential()
    model.add(Masking(mask_value= -1, input_shape=(1000, 64)))
    model.add(LSTM(128, return_sequences=True))
#     model.add(LSTM(128))
    model.add(Dense(64, activation='relu'))
#     model.add(Dropout(0.1))
    model.add(Dense(64, activation='sigmoid'))
    model.summary()
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

In [36]:
def create_model_NER():
    model = Sequential()
    model.add(Masking(mask_value = -1, input_shape=(64, 1000)))
    # 双向RNN包装器
    model.add(Bidirectional(LSTM(128 // 2, return_sequences=True)))
    crf = CRF(2, sparse_target=True)
    model.add(crf)
    model.summary()
    model.compile('adam', loss=crf.loss_function, metrics=[crf.accuracy])
    return model

In [49]:
def create_model_NER_2():
    inputs = Input(shape=(64, 1000))
    x = Masking(mask_value = -1)(inputs)
    x = Bidirectional(LSTM(15,return_sequences=True))(x)
    x = Lambda(lambda x: x, output_shape=lambda s: s)(x)
    x = Conv1D(15, 3, padding='same', activation='relu')(x)
    x = TimeDistributed(Dense(2))(x)
    crf = CRF(units=2, sparse_target=True)
    outputs = crf(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.summary()
    model.compile(optimizer='adam',
                  loss=crf.loss_function,
                  metrics=[crf.accuracy]
#                   loss=losses.crf_loss,
#                   metrics=[metrics.crf_accuracy]
                 )
    return model

In [38]:
def create_model_CNNLSTM():
    inputs = Input(shape=(64, 1000))
    x = Masking(mask_value = -1)(inputs)
    x = Bidirectional(LSTM(64,return_sequences=True))(x)
    x = Lambda(lambda x: x, output_shape=lambda s: s)(x)
    x = Conv1D(15, 3, padding='same', activation='relu')(x)
    x = TimeDistributed(Dense(2))(x)
#     outputs = CRF(units=2, sparse_target=True)(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.summary()
    model.compile(optimizer='adam',
              loss=losses.crf_loss,
              metrics=[metrics.crf_accuracy]
             )
#     model.compile(optimizer='adam',
#                   loss=losses.crf_loss,
#                   metrics=[metrics.crf_accuracy]
#                  )
    return model

In [39]:
def create_model_lstm():
    model = Sequential()
    model.add(Masking(mask_value= -1, input_shape=(1000, 64)))
    model.add(LSTM(128))
    model.add(Dense(64, activation='relu'))
#     model.add(Dropout(0.1))
    model.add(Dense(64, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.summary()
    return model

In [183]:
import keras
from keras import backend as K
import tensorflow as tf
from keras.optimizers import Adam

# Define our custom loss function
def focal_loss(y_true, y_pred):
    gamma = 2.0
    alpha = 0.25
    pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
    pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
    return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))

# Compile our model
# adam = Adam(lr=0.0001)
# model.compile(loss=[focal_loss], metrics=["accuracy"], optimizer=adam)


In [180]:
# def LSTM_CNN_Model():
# #     inputs = Input(shape=(64, 1000))
#     inputs = Input(shape=(1000, 64))
# #     x = Masking(mask_value = -1)(inputs)
# #     x = LSTM(64, return_sequences=True)(inputs)
# #     x = Lambda(lambda x: x, output_shape=lambda s: s)(x)
#     x= Conv1D(256, 5, padding='same')(inputs)
#     x = MaxPooling1D(3, 3, padding='same')(x)
#     x = Conv1D(128, 5, padding='same')(x)
#     x = MaxPooling1D(3, 3, padding='same')(x)
#     x = Conv1D(64, 3, padding='same')(x)
#     x = Flatten()(x)
#     x = Dropout(0.1)(x)
#     x = BatchNormalization()(x)
#     x = Dense(256, activation='relu')(x)
#     x = Dropout(0.1)(x)
#     outputs = Dense(64, activation='sigmoid')(x)
#     model = Model(inputs=inputs, outputs=outputs)
# #     focal_loss bias_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)
#     adam = Adam(lr=0.0001)
# #     model.compile(loss=[focal_loss], metrics=["accuracy"], optimizer=adam)
#     model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
#     model.summary()
#     return model

In [187]:
# def LSTM_CNN_Model():
#     inputs = Input(shape=(1000, 64))
#     x= Conv1D(256, 5, padding='same')(inputs)
#     x = MaxPooling1D(3, 3, padding='same')(x)
#     x = Conv1D(128, 5, padding='same')(x)
#     x = MaxPooling1D(3, 3, padding='same')(x)
#     x = Conv1D(64, 3, padding='same')(x)
#     x = Flatten()(x)
#     x = Dropout(0.1)(x)
#     x = BatchNormalization()(x)
#     x = Dense(256, activation='relu',bias_initializer=keras.initializers.Constant(value=-2))(x)
#     x = Dropout(0.1)(x)
#     outputs = Dense(64, activation='sigmoid')(x)
#     model = Model(inputs=inputs, outputs=outputs)
# #     focal_loss 
#     adam = Adam(lr=0.0001)
#     model.compile(loss=[focal_loss], metrics=["accuracy"], optimizer=adam)
# #     model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
#     model.summary()
#     return model

In [1]:
def LSTM_CNN_Model():
    inputs = Input(shape=(64, 1000))
    x= Conv1D(256, 5, padding='same')(inputs)
    x = MaxPooling1D(3, 3, padding='same')(x)
    x = Conv1D(128, 5, padding='same')(x)
    x = MaxPooling1D(3, 3, padding='same')(x)
    x = Conv1D(64, 3, padding='same')(x)
    x = Flatten()(x)
    x = Dropout(0.1)(x)
    x = BatchNormalization()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.1)(x)
    outputs = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=inputs, outputs=outputs)
    adam = Adam(lr=0.0001)
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
LSTM_CNN_Modelmodel = 

In [2]:
train_data

NameError: name 'train_data' is not defined

In [ ]:
# def LSTM_CNN_Model():
#     #构建CNN分类模型(LeNet-5)
#     #模型结构：嵌入-卷积池化*2-dropout-BN-全连接-dropout-全连接
# #     inputs = Input(shape=(64, 1000))
#     inputs = Input(shape=(1000, 64))
# #     x = Masking(mask_value = -1)(inputs)
#     x = LSTM(64, return_sequences=True)(inputs)
# #     x = Lambda(lambda x: x, output_shape=lambda s: s)(x)
#     x= Conv1D(256, 5, padding='same')(x)
#     x = MaxPooling1D(3, 3, padding='same')(x)
#     x = Conv1D(128, 5, padding='same')(x)
#     x = MaxPooling1D(3, 3, padding='same')(x)
#     x = Conv1D(64, 3, padding='same')(x)
#     x = Flatten()(x)
#     x = Dropout(0.1)(x)
#     x = BatchNormalization()(x)
#     x = Dense(256, activation='relu')(x)
#     x = Dropout(0.1)(x)
#     outputs = Dense(64, activation='sigmoid')(x)
#     model = Model(inputs=inputs, outputs=outputs)
# #     focal_loss
#     adam = Adam(lr=0.0001)
#     model.compile(loss=[focal_loss], metrics=["accuracy"], optimizer=adam)
# #     model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
#     model.summary()
#     return model

In [63]:
def model_bilstm():
    model = Sequential()
    model.add(Masking(mask_value= -1, input_shape=(64,1000)))
    model.add(Bidirectional(LSTM(30 // 2, return_sequences=True)))
    model.add(Dense(2, activation='softmax'))
    model.summary()
#     model.add(Dense(1, activation='sigmoid'))
#     model.summary()
#     model.compile('adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [154]:
def model_lstm_cnn():
    model = Sequential()
    model.add(Bidirectional(LSTM(15 // 2, return_sequences=True), input_shape=(64,1000)))
    model.add(Conv1D(15, 3, padding='same'))
#     model.add(MaxPooling1D(3, 3, padding='same'))
    model.add(Dense(2, activation='relu'))
#     model.add(Dense(2, activation='softmax'))
#     model.compile('adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.add(Dense(1, activation='sigmoid'))
    adam = Adam(lr=0.0001)
    model.compile('adam', loss=[focal_loss], metrics=['accuracy'])
    #     model.compile('adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
#     model.add(Dense(1, activation='sigmoid'))
#     model.summary()
#     model.compile('adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [171]:
def model_lstm_cnn_T():
    model = Sequential()
    model.add(Conv1D(64, 3, padding='same', input_shape=(1000,64)))
    model.add(MaxPooling1D(3, 3, padding='same'))
    model.add(Bidirectional(LSTM(64 // 2, return_sequences=False)))
#     model.add(Conv1D(15, 3, padding='same'))
#     model.add(MaxPooling1D(3, 3, padding='same'))
    model.add(Dense(64, activation='relu'))
#     model.add(Dense(2, activation='softmax'))
#     model.compile('adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.add(Dense(64, activation='sigmoid'))
    adam = Adam(lr=0.0001)
    model.compile('adam', loss=[focal_loss], metrics=['accuracy'])
    #     model.compile('adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
#     model.add(Dense(1, activation='sigmoid'))
#     model.summary()
#     model.compile('adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [191]:
model = LSTM_CNN_Model()

Model: "model_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        (None, 64, 1000)          0         
_________________________________________________________________
conv1d_61 (Conv1D)           (None, 64, 256)           1280256   
_________________________________________________________________
max_pooling1d_34 (MaxPooling (None, 22, 256)           0         
_________________________________________________________________
conv1d_62 (Conv1D)           (None, 22, 128)           163968    
_________________________________________________________________
max_pooling1d_35 (MaxPooling (None, 8, 128)            0         
_________________________________________________________________
conv1d_63 (Conv1D)           (None, 8, 64)             24640     
_________________________________________________________________
flatten_17 (Flatten)         (None, 512)               0  

In [201]:
# t_train_labels

In [198]:
model.fit(train_data_lstm, train_labels, epochs=5, batch_size=100)

ValueError: Error when checking target: expected dense_59 to have 2 dimensions, but got array with shape (1089, 64, 1)

In [100]:
cal_score(model, train_data, train_labels)
cal_score(model, test_data, test_labels)

数据集:
预测正确数量,训练集样本量:
69588 69696
训练集精确度等指标：
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     64572
         1.0       1.00      0.98      0.99      5124

    accuracy                           1.00     69696
   macro avg       1.00      0.99      0.99     69696
weighted avg       1.00      1.00      1.00     69696

混淆矩阵：
[[64549    23]
 [   85  5039]]
数据集:
预测正确数量,训练集样本量:
8826 9600
训练集精确度等指标：
              precision    recall  f1-score   support

         0.0       0.96      0.95      0.96      9087
         1.0       0.28      0.31      0.29       513

    accuracy                           0.92      9600
   macro avg       0.62      0.63      0.63      9600
weighted avg       0.92      0.92      0.92      9600

混淆矩阵：
[[8665  422]
 [ 352  161]]


In [63]:
# model = create_model_NER()

In [64]:
# model = create_model_NER_2()

In [95]:
# model = model_bilstm()

In [84]:
train_labels_del_lstm = keras.utils.to_categorical(train_labels_del, 3)

In [85]:
train_labels_del_lstm[2][:,1],train_labels_del[2]

(array([0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32),
 array([ 0.,  1.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.]))

In [76]:
train_labels_del[0]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.])

In [67]:
model.fit(train_data_del_lstm, train_labels_del_lstm, epochs=5, batch_size=100)

Epoch 1/5
1089/1089 [==============================] - 11s 10ms/step - loss: 0.4341 - acc: 0.8489
Epoch 2/5
1089/1089 [==============================] - 4s 4ms/step - loss: 0.4056 - acc: 0.8516
Epoch 3/5
 100/1089 [=>............................] - ETA: 2s - loss: 0.4054 - acc: 0.8469

D:\--storage--\program\application\windows-installer\Miniconda3\Miniconda3\envs\Commonl_Py3.6\lib\site-packages\keras\callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.306165). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


1089/1089 [==============================] - 3s 3ms/step - loss: 0.3905 - acc: 0.8517
Epoch 4/5
1089/1089 [==============================] - 3s 2ms/step - loss: 0.3788 - acc: 0.8514
Epoch 5/5
1089/1089 [==============================] - 2s 2ms/step - loss: 0.3666 - acc: 0.8585


In [52]:
train_labels_lstm = keras.utils.to_categorical(train_labels, 2)

In [73]:
train_labels_lstm[0]

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.

In [53]:
model.fit(train_data_lstm, train_labels_lstm, epochs=5, batch_size=100)

Epoch 1/5
1089/1089 [==============================] - 8s 7ms/step - loss: 1.1006 - crf_viterbi_accuracy: 0.2751
Epoch 2/5
1089/1089 [==============================] - 2s 2ms/step - loss: 0.5749 - crf_viterbi_accuracy: 0.6196
Epoch 3/5
1089/1089 [==============================] - 2s 2ms/step - loss: 0.4468 - crf_viterbi_accuracy: 0.7483
Epoch 4/5
1089/1089 [==============================] - 2s 2ms/step - loss: 0.3591 - crf_viterbi_accuracy: 0.8402
Epoch 5/5
1089/1089 [==============================] - 2s 2ms/step - loss: 0.2978 - crf_viterbi_accuracy: 0.9162


In [86]:
temp = model.predict(train_data_del_lstm)

In [92]:
# temp[2]

In [91]:
# [np.argmax(t) for t in temp[2]]

In [93]:
labels = []
preds = []
for i in range(len(train_labels)):
    for j in range(64):
        if train_labels[i][j] == 1:
            res = [np.argmax(t) for t in temp[i]]
            preds.append(res[j])

In [94]:
preds.count(1),len(preds),preds.count(1)/len(preds)

(89, 5124, 0.017369242779078843)

In [135]:
model.fit(train_data_del_new, train_labels_del_new, epochs=5, batch_size=100)

Epoch 1/5
1089/1089 [==============================] - 22s 20ms/step - loss: -5.0505 - acc: 0.2911
Epoch 2/5
1089/1089 [==============================] - 18s 16ms/step - loss: -8.1365 - acc: 0.3586
Epoch 3/5
1089/1089 [==============================] - 18s 17ms/step - loss: -8.2289 - acc: 0.3637
Epoch 4/5
1089/1089 [==============================] - 17s 15ms/step - loss: -8.3003 - acc: 0.3635
Epoch 5/5
1089/1089 [==============================] - 17s 15ms/step - loss: -8.4244 - acc: 0.3591


In [59]:
model.fit(train_data, train_labels, epochs=5, batch_size=100)

In [120]:
model.fit(train_data_del, train_labels_del, epochs=5, batch_size=100)

Epoch 1/5
1089/1089 [==============================] - 20s 18ms/step - loss: -5.6334 - acc: 0.2672
Epoch 2/5
1089/1089 [==============================] - 17s 16ms/step - loss: -8.8750 - acc: 0.3330
Epoch 3/5
1089/1089 [==============================] - 15s 14ms/step - loss: -8.9546 - acc: 0.3338
Epoch 4/5
1089/1089 [==============================] - 15s 14ms/step - loss: -9.0215 - acc: 0.3298
Epoch 5/5
1089/1089 [==============================] - 15s 14ms/step - loss: -9.0283 - acc: 0.3311


In [110]:
model = model_bilstm()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_19 (Masking)         (None, 64, 1000)          0         
_________________________________________________________________
bidirectional_19 (Bidirectio (None, 64, 30)            121920    
_________________________________________________________________
dense_16 (Dense)             (None, 64, 1)             31        
Total params: 121,951
Trainable params: 121,951
Non-trainable params: 0
_________________________________________________________________


In [111]:
# train_data_lstm

In [112]:
train_labels_del_3 = np.array(np.expand_dims(train_labels_del, axis=2))

In [113]:
type(train_labels_del_3)

numpy.ndarray

In [121]:
model.fit(train_data_lstm, train_labels_del_3, epochs=50, batch_size=100)

Epoch 1/50
1089/1089 [==============================] - 3s 2ms/step - loss: -7.9073 - crf_viterbi_accuracy: 0.3550
Epoch 2/50
1089/1089 [==============================] - 3s 2ms/step - loss: -9.6664 - crf_viterbi_accuracy: 0.3563
Epoch 3/50
1089/1089 [==============================] - 3s 2ms/step - loss: -11.8790 - crf_viterbi_accuracy: 0.3576
Epoch 4/50
1089/1089 [==============================] - 3s 3ms/step - loss: -14.2077 - crf_viterbi_accuracy: 0.3584
Epoch 5/50
1089/1089 [==============================] - 3s 3ms/step - loss: -16.8970 - crf_viterbi_accuracy: 0.3594
Epoch 6/50
1089/1089 [==============================] - 3s 3ms/step - loss: -20.1754 - crf_viterbi_accuracy: 0.3602
Epoch 7/50
1089/1089 [==============================] - 3s 3ms/step - loss: -23.6390 - crf_viterbi_accuracy: 0.3602
Epoch 8/50
1089/1089 [==============================] - 3s 3ms/step - loss: -27.5753 - crf_viterbi_accuracy: 0.3627
Epoch 9/50
1089/1089 [==============================] - 3s 3ms/step - loss

In [99]:
def cal_score(model, train_data, train_labels):
    print("数据集:")
    predict_target_msb_prob = model.predict(train_data)
    predict_target_msb_label = (predict_target_msb_prob > 0.5).astype(int)
    predict_target_msb_1D = predict_target_msb_label.flatten()
    train_labels_1D = train_labels.flatten()
    print("预测正确数量,训练集样本量:")
    print(sum(predict_target_msb_1D == train_labels_1D),len(train_labels_1D))
    print("训练集精确度等指标：")
    print(metrics.classification_report(train_labels_1D,predict_target_msb_1D))
    print("混淆矩阵：")
    print(metrics.confusion_matrix(train_labels_1D,predict_target_msb_1D))

In [122]:
predict_target_msb_prob = model.predict(train_data_lstm)
predict_target_msb_label = (predict_target_msb_prob > 0.5).astype(int)
predict_target_msb_label

array([[[1, 0],
        [1, 0],
        [1, 0],
        ...,
        [1, 0],
        [1, 0],
        [1, 0]],

       [[1, 0],
        [1, 0],
        [1, 0],
        ...,
        [1, 0],
        [1, 0],
        [1, 0]],

       [[1, 0],
        [0, 1],
        [1, 0],
        ...,
        [1, 0],
        [1, 0],
        [1, 0]],

       ...,

       [[1, 0],
        [1, 0],
        [0, 1],
        ...,
        [1, 0],
        [1, 0],
        [1, 0]],

       [[1, 0],
        [1, 0],
        [1, 0],
        ...,
        [1, 0],
        [1, 0],
        [1, 0]],

       [[1, 0],
        [1, 0],
        [1, 0],
        ...,
        [1, 0],
        [1, 0],
        [1, 0]]])

In [138]:
temp = model.predict(train_data_lstm)

In [ ]:
train_data_lstm

In [139]:
len(temp[0][:,1])

64

In [142]:
temp.shape

(1089, 64, 2)

In [140]:
temp[0][:,1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [141]:
temp[1][:,1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [136]:
test_labels_del

array([[ 0.,  0.,  0., ..., -1., -1., -1.],
       [ 0.,  0.,  0., ..., -1., -1., -1.],
       [ 1.,  1.,  0., ..., -1., -1., -1.],
       ...,
       [ 0.,  1.,  0., ..., -1., -1., -1.],
       [ 1.,  1.,  0., ..., -1., -1., -1.],
       [ 0.,  0.,  0., ..., -1., -1., -1.]])

In [133]:
def cal_score_lstm(model, train_data, train_labels):
    print("数据集:")
    predict_target_msb_prob = model.predict(train_data)
#     predict_target_msb_label = (predict_target_msb_prob > 0.5).astype(int)
    predict_target_msb_label = predict_target_msb_prob[0][:,1]
    predict_target_msb_1D = predict_target_msb_label.flatten()
    train_labels_1D = train_labels.flatten()
    print("预测正确数量,训练集样本量:")
    print(sum(predict_target_msb_1D == train_labels_1D),len(train_labels_1D))
    print("训练集精确度等指标：")
    print(metrics.classification_report(train_labels_1D,predict_target_msb_1D))
    print("混淆矩阵：")
    print(metrics.confusion_matrix(train_labels_1D,predict_target_msb_1D))

In [134]:
test_labels_del_3 = np.array(np.expand_dims(test_labels_del, axis=2))
cal_score_lstm(model, train_data_lstm, train_labels_del)
cal_score_lstm(model, test_data_lstm, test_labels_del)

数据集:
预测正确数量,训练集样本量:


C:\Users\Linever\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:9: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if __name__ == '__main__':


TypeError: 'bool' object is not iterable

In [123]:
cal_score(model, train_data, train_labels)
cal_score(model, test_data, test_labels)

数据集:
预测正确数量,训练集样本量:
65068 69696
训练集精确度等指标：
              precision    recall  f1-score   support

         0.0       0.94      0.99      0.96     64572
         1.0       0.61      0.27      0.38      5124

    accuracy                           0.93     69696
   macro avg       0.78      0.63      0.67     69696
weighted avg       0.92      0.93      0.92     69696

混淆矩阵：
[[63662   910]
 [ 3718  1406]]
数据集:
预测正确数量,训练集样本量:
8939 9600
训练集精确度等指标：
              precision    recall  f1-score   support

         0.0       0.96      0.97      0.96      9087
         1.0       0.32      0.26      0.29       513

    accuracy                           0.93      9600
   macro avg       0.64      0.62      0.63      9600
weighted avg       0.92      0.93      0.93      9600

混淆矩阵：
[[8805  282]
 [ 379  134]]


In [122]:
cal_score(model, train_data_del, train_labels_del)
cal_score(model, test_data_del, test_labels_del)

数据集:
预测正确数量,训练集样本量:
23745 69696
训练集精确度等指标：


D:\--storage--\program\application\windows-installer\Miniconda3\Miniconda3\envs\Commonl_Py3.6\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\--storage--\program\application\windows-installer\Miniconda3\Miniconda3\envs\Commonl_Py3.6\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\--storage--\program\application\windows-installer\Miniconda3\Miniconda3\envs\Commonl_Py3.6\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 

              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00     41832
         0.0       0.35      1.00      0.52     23733
         1.0       0.01      0.01      0.01      4131

    accuracy                           0.34     69696
   macro avg       0.12      0.34      0.18     69696
weighted avg       0.12      0.34      0.18     69696

混淆矩阵：
[[    0 39976  1856]
 [    0 23717    16]
 [    0  4103    28]]
数据集:
预测正确数量,训练集样本量:
2108 9600
训练集精确度等指标：
              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00      6959
         0.0       0.23      1.00      0.37      2108
         1.0       0.00      0.00      0.00       533

    accuracy                           0.22      9600
   macro avg       0.08      0.33      0.12      9600
weighted avg       0.05      0.22      0.08      9600

混淆矩阵：
[[   0 6686  273]
 [   0 2108    0]
 [   0  533    0]]


D:\--storage--\program\application\windows-installer\Miniconda3\Miniconda3\envs\Commonl_Py3.6\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\--storage--\program\application\windows-installer\Miniconda3\Miniconda3\envs\Commonl_Py3.6\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\--storage--\program\application\windows-installer\Miniconda3\Miniconda3\envs\Commonl_Py3.6\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 

In [97]:
cal_score(model, train_data_lstm, train_labels_3)
cal_score(model, test_data_lstm, test_labels_3)

In [187]:
def CNN_Model(train_data, train_labels, test_data, test_labels):
    #构建CNN分类模型(LeNet-5)
    #模型结构：嵌入-卷积池化*2-dropout-BN-全连接-dropout-全连接
    model_msb = Sequential()
    model_msb.add(Conv1D(256, 5, padding='same', input_shape = (1000,64)))
    model_msb.add(MaxPooling1D(3, 3, padding='same'))
    model_msb.add(Conv1D(128, 5, padding='same'))
    model_msb.add(MaxPooling1D(3, 3, padding='same'))
    model_msb.add(Conv1D(64, 3, padding='same'))
    model_msb.add(Flatten())
    model_msb.add(Dropout(0.1))
    model_msb.add(BatchNormalization())  # (批)规范化层
    model_msb.add(Dense(256, activation='relu'))
    model_msb.add(Dropout(0.1))
    model_msb.add(Dense(64, activation='sigmoid'))
    # model_msb.compile(loss=my_loss,optimizer='adam',metrics=['accuracy'])
    model_msb.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    # one_hot_labels = keras.utils.to_categorical(y_train, num_classes=3)  # 将标签转换为one-hot编码
    model_msb.summary()
    
    model_msb.fit(train_data, train_labels,epochs=10, batch_size=10)
    
    print("训练集:")
    predict_target_msb_prob = model_msb.predict(train_data)
    predict_target_msb_label = (predict_target_msb_prob > 0.5).astype(int)
    predict_target_msb_1D = predict_target_msb_label.flatten()
    train_labels_1D = train_labels.flatten()
    print("预测正确数量,训练集样本量:")
    print(sum(predict_target_msb_1D == train_labels_1D),len(train_labels_1D))
    print("训练集精确度等指标：")
    print(metrics.classification_report(train_labels_1D,predict_target_msb_1D))
    print("混淆矩阵：")
    print(metrics.confusion_matrix(train_labels_1D,predict_target_msb_1D))

    print("测试集:")
    predict_target_test_msb_prob =model_msb.predict(test_data)
    predict_target_test_msb_label = (predict_target_test_msb_prob > 0.5).astype(int)
    predict_target_test_msb_1D = predict_target_test_msb_label.flatten()
    test_labels_1D = test_labels.flatten()
    print("预测正确数量,训练集样本量:")
    print(sum(predict_target_test_msb_1D == test_labels_1D),len(test_labels_1D))
    print("测试集精确度等指标：")
    print(metrics.classification_report(test_labels_1D,predict_target_test_msb_1D))
    print("混淆矩阵：")
    print(metrics.confusion_matrix(test_labels_1D,predict_target_test_msb_1D))

In [188]:
CNN_Model(train_data, train_labels, test_data, test_labels)






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 1000, 256)         82176     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 334, 256)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 334, 128)          163968    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 112, 128)          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 112, 64)           24640     
____________________________________

In [190]:
#构建TextCNN模型
#模型结构：词嵌入-卷积池化*3-拼接-全连接-dropout-全连接
def TextCNN_model_1(train_data, train_labels, test_data, test_labels):
    main_input = Input(shape=(1000,64))
    # 词窗大小分别为3,4,5
    cnn1 = Conv1D(128, 4, padding='same', strides=1, activation='relu')(main_input)
    cnn1 = MaxPooling1D(pool_size=64)(cnn1)
    cnn2 = Conv1D(128, 8, padding='same', strides=1, activation='relu')(main_input)
    cnn2 = MaxPooling1D(pool_size=64)(cnn2)
    cnn3 = Conv1D(128, 16, padding='same', strides=1, activation='relu')(main_input)
    cnn3 = MaxPooling1D(pool_size=64)(cnn3)
    # 合并三个模型的输出向量
    cnn = concatenate([cnn1, cnn2, cnn3], axis=-1)
    flat = Flatten()(cnn)
    drop = Dropout(0.2)(flat)
    main_output = Dense(64, activation='sigmoid')(drop)
    model = Model(inputs=main_input, outputs=main_output)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()

    K.set_value(model.optimizer.lr,1e-4)
    model.fit(train_data, train_labels,epochs=5, batch_size=10)
    
    print("训练集:")
    predict_target_msb_prob = model.predict(train_data)
    predict_target_msb_label = (predict_target_msb_prob > 0.5).astype(int)
    predict_target_msb_1D = predict_target_msb_label.flatten()
    train_labels_1D = train_labels.flatten()
    print("预测正确数量,训练集样本量:")
    print(sum(predict_target_msb_1D == train_labels_1D),len(train_labels_1D))
    print("训练集精确度等指标：")
    print(metrics.classification_report(train_labels_1D,predict_target_msb_1D))
    print("混淆矩阵：")
    print(metrics.confusion_matrix(train_labels_1D,predict_target_msb_1D))

    print("测试集:")
    predict_target_test_msb_prob =model.predict(test_data)
    predict_target_test_msb_label = (predict_target_test_msb_prob > 0.5).astype(int)
    predict_target_test_msb_1D = predict_target_test_msb_label.flatten()
    test_labels_1D = test_labels.flatten()
    print("预测正确数量,训练集样本量:")
    print(sum(predict_target_test_msb_1D == test_labels_1D),len(test_labels_1D))
    print("测试集精确度等指标：")
    print(metrics.classification_report(test_labels_1D,predict_target_test_msb_1D))
    print("混淆矩阵：")
    print(metrics.confusion_matrix(test_labels_1D,predict_target_test_msb_1D))

In [191]:
TextCNN_model_1(train_data, train_labels, test_data, test_labels)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1000, 64)     0                                            
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 1000, 128)    32896       input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 1000, 128)    65664       input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 1000, 128)    131200      input_1[0][0]                    
____________________________________________________________________________________________

D:\--storage--\program\application\windows-installer\Miniconda3\Miniconda3\envs\Commonl_Py3.6\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\--storage--\program\application\windows-installer\Miniconda3\Miniconda3\envs\Commonl_Py3.6\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\--storage--\program\application\windows-installer\Miniconda3\Miniconda3\envs\Commonl_Py3.6\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 

In [53]:
def my_loss(y_true, y_pred):
#     temp = lambda(x )
    temp = y_pred
    right_count = 0
    for i in range(64):
        if(y_true[i] == 1):
            if(temp[i] == 1):
                right_count = right_count + 1
    return right_count/64

In [194]:
def multilabel_categorical_crossentropy(y_true, y_pred):
    """多标签分类的交叉熵
    说明：y_true和y_pred的shape一致，y_true的元素非0即1，
         1表示对应的类为目标类，0表示对应的类为非目标类。
    """
#     print("y_pred:")
#     print(y_pred.shape)
#     print("y_true:")
#     print(y_true.shape)
    y_pred = (1 - 2 * y_true) * y_pred
#     print("y_pred:")
#     print(y_pred.shape)
    y_pred_neg = y_pred - y_true * 1e12
    y_pred_pos = y_pred - (1 - y_true) * 1e12
    zeros = K.zeros_like(y_pred[..., :1])
    y_pred_neg = K.concatenate([y_pred_neg, zeros], axis=-1)
    y_pred_pos = K.concatenate([y_pred_pos, zeros], axis=-1)
    neg_loss = K.logsumexp(y_pred_neg, axis=-1)
    pos_loss = K.logsumexp(y_pred_pos, axis=-1)
    print(pos_loss.shape)
    return neg_loss + pos_loss

In [197]:
#构建TextCNN模型
#模型结构：词嵌入-卷积池化*3-拼接-全连接-dropout-全连接
def TextCNN_model_2(train_data, train_labels, test_data, test_labels):
    main_input = Input(shape=(1000,64))
    # 词窗大小分别为3,4,5
    cnn1 = Conv1D(128, 4, padding='same', strides=1, activation='relu')(main_input)
    cnn1 = MaxPooling1D(pool_size=64)(cnn1)
    cnn2 = Conv1D(128, 8, padding='same', strides=1, activation='relu')(main_input)
    cnn2 = MaxPooling1D(pool_size=64)(cnn2)
    cnn3 = Conv1D(128, 16, padding='same', strides=1, activation='relu')(main_input)
    cnn3 = MaxPooling1D(pool_size=64)(cnn3)
    # 合并三个模型的输出向量
    cnn = concatenate([cnn1, cnn2, cnn3], axis=-1)
    flat = Flatten()(cnn)
    drop = Dropout(0.2)(flat)
    main_output = Dense(64, activation='softmax')(drop)
    model = Model(inputs=main_input, outputs=main_output)
    model.compile(loss=multilabel_categorical_crossentropy, optimizer='adam', metrics=['accuracy'])
    model.summary()

    K.set_value(model.optimizer.lr,1e-4)
    model.fit(train_data, train_labels,epochs=10, batch_size=10)
    
    print("训练集:")
    predict_target_msb_prob = model.predict(train_data)
    predict_target_msb_label = (predict_target_msb_prob > 0.5).astype(int)
    predict_target_msb_1D = predict_target_msb_label.flatten()
    train_labels_1D = train_labels.flatten()
    print("预测正确数量,训练集样本量:")
    print(sum(predict_target_msb_1D == train_labels_1D),len(train_labels_1D))
    print("训练集精确度等指标：")
    print(metrics.classification_report(train_labels_1D,predict_target_msb_1D))
    print("混淆矩阵：")
    print(metrics.confusion_matrix(train_labels_1D,predict_target_msb_1D))

    print("测试集:")
    predict_target_test_msb_prob =model.predict(test_data)
    predict_target_test_msb_label = (predict_target_test_msb_prob > 0.5).astype(int)
    predict_target_test_msb_1D = predict_target_test_msb_label.flatten()
    test_labels_1D = test_labels.flatten()
    print("预测正确数量,训练集样本量:")
    print(sum(predict_target_test_msb_1D == test_labels_1D),len(test_labels_1D))
    print("测试集精确度等指标：")
    print(metrics.classification_report(test_labels_1D,predict_target_test_msb_1D))
    print("混淆矩阵：")
    print(metrics.confusion_matrix(test_labels_1D,predict_target_test_msb_1D))

In [198]:
TextCNN_model_2(train_data, train_labels, test_data, test_labels)

(?,)
Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 1000, 64)     0                                            
__________________________________________________________________________________________________
conv1d_13 (Conv1D)              (None, 1000, 128)    32896       input_4[0][0]                    
__________________________________________________________________________________________________
conv1d_14 (Conv1D)              (None, 1000, 128)    65664       input_4[0][0]                    
__________________________________________________________________________________________________
conv1d_15 (Conv1D)              (None, 1000, 128)    131200      input_4[0][0]                    
_______________________________________________________________________________________

In [40]:
# lsb边界是否识别正确的准确度统计

right_count = 0
total_count = 0
for i in range(len(test_labels_1D)):
    if(test_labels_1D[i] == 1):
        total_count = total_count + 1
        if(predict_target_test_msb_1D[i] == 1):
            right_count = right_count + 1
acc = right_count / total_count

In [41]:
acc

0.3140814081408141

In [48]:
def get_locate_label_array(test_label_array):
    test_label_locate_array = []
    for id in range(len(test_label_array)):
        temp_list = test_label_array[id]
        cur_label = []
        for i in range(64):
            if(temp_list[i] == 1):
                cur_label.append(i)
        test_label_locate_array.append(cur_label)
    return test_label_locate_array

In [49]:
final_label_array = get_locate_label_array(predict_target_test_msb_label)
test_label_locate_array = get_locate_label_array(test_labels)

In [50]:
for id in range(len(final_label_array)):
    print(final_label_array[id])
    print(test_label_locate_array[id])
    print("------------------------------------------")

[59, 63]
[23, 39]
------------------------------------------
[15, 31, 47, 55, 59, 63]
[15, 27, 41, 42, 59, 63]
------------------------------------------
[7, 47, 63]
[15, 23]
------------------------------------------
[59, 63]
[23, 39]
------------------------------------------
[39, 41, 47, 59, 63]
[15]
------------------------------------------
[39, 59, 63]
[7, 31, 32, 33, 38, 39, 49, 50, 55, 59, 63]
------------------------------------------
[]
[15]
------------------------------------------
[39, 63]
[15, 31, 59, 63]
------------------------------------------
[39, 59, 63]
[15, 31, 59, 63]
------------------------------------------
[7, 15, 31, 47, 59, 63]
[15, 23]
------------------------------------------
[39, 59, 63]
[15, 31, 59, 63]
------------------------------------------
[]
[15]
------------------------------------------
[7, 8, 15, 31, 47, 49, 55, 59, 63]
[55]
------------------------------------------
[15, 47, 55, 59, 63]
[23, 39]
------------------------------------------
[39